In [11]:
from exchange.log import LogLoad

In [8]:
loader = LogLoad()

loader.append('../DATA/BB/05/BB-2020-05-09T06-16-23.367761Z.log.gz')

print(loader.log_data.head())

../DATA/BB/05/BB-2020-05-09T06-16-23.367761Z.log.gz 2020050906-0522--2020050906-0533
   action              time  sequence    price  volume  checksum
0       1  1589004982636752         0      NaN     NaN       NaN
1       3  1589004982636752         0  96020.0   0.025       NaN
2       3  1589004982636752         1  96050.0   0.045       NaN
3       3  1589004982636752         2  96060.0   4.100       NaN
4       3  1589004982636752         3  96065.0   0.025       NaN


In [10]:
loader.log_data
loader.update_logs()

loader.ask_log
loader.bit_log




,action,time,sequence,price,volume,checksum
